## 데이터 불러오기

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip install sentence_transformers

In [4]:
# pip install textblob

In [5]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

import nltk
from nltk.tokenize import RegexpTokenizer       
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4") 

FHP_article = pd.read_json('healthcare_project_FHP_article.json')
FHP_article = FHP_article.loc[:,'title':]
FHP_article

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,title,date,abstract
0,Effects of scapular treatment on chronic neck ...,2024 Apr 1,Background: Chronic neck pain (CNP) is a commo...
1,Association between smartphone addiction and m...,2024 Apr 1,Background: The purpose of this study was to c...
2,Effects of Telerehabilitation Combining Diaphr...,2024 Mar 11,Background: Forward head posture and rounded s...
3,Effects of Myofascial Release Technique along ...,2024 Mar 4,The purpose of this randomized controlled tria...
4,Sensory Organization and Postural Control Stra...,2024 Mar 26,Objective: This study aimed to compare sensory...
...,...,...,...
1246,Mechanism of production of gait unsteadiness b...,1975 Feb,The hypothesis is put forward that the disturb...
1247,Effect of forward head inclination on visual o...,1972 Nov,
1248,[The mechanism of elongation of the eye-axis. ...,1972 Aug,
1249,Rheumatoid arthritis--atlanto-axial subluxatio...,1966 Mar 5,"The clinical picture, radiological findings an..."


## 데이터 전처리

### 결측치 처리 
- abstract가 없는 값은 제목으로 대체

In [6]:
FHP_article.query("abstract==''").count()

title       9
date        9
abstract    9
dtype: int64

In [7]:
FHP_article.loc[FHP_article.query("abstract==''").index,'abstract'] = FHP_article.loc[FHP_article.query("abstract==''").index,'title']

In [8]:
FHP_article.query("abstract==''").sum()

title       0
date        0
abstract    0
dtype: object

### 발행일 변환

In [9]:
def date_year(raw):
    date_format = raw.split(" ")[0]
    return date_format
def date_month(raw):
    if len(raw.split(" ")) >= 2:
        date_format = raw.split(" ")[1]
    else:
        date_format = None
    return date_format
def date_date(raw):
    if len(raw.split(" ")) >= 3:
        date_format = raw.split(" ")[2]
    else:
        date_format = None
    return date_format
def word_split(raw,word_list):
    word_list.append(raw.split())
    return word_list

In [10]:
FHP_article.loc[:,'date_year']=FHP_article['date'].apply(date_year)
FHP_article.loc[:,'date_month']=FHP_article['date'].apply(date_month)
FHP_article.loc[:,'date_date']=FHP_article['date'].apply(date_date)

### 정규화 : 대소문자 통합

In [11]:
def word_lower(raw):
    abstract_list = raw.split()
    modified_list = []
    for abstract_word in abstract_list :
         modified_list.append(abstract_word.lower())
    return " ".join(modified_list)

In [12]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract'].apply(word_lower)

In [51]:
FHP_article.loc[2,'abstract_clean']

'  forward head posture and rounded shoulder posture are common postural variants found in upper crossed syndrome which can lead to limited neck mobility respiratory problems and other issues the purpose of this study was to investigate the effects of telerehabilitation combining diaphragmatic breathing reeducation and shoulder stabilization exercises on young men with upper crossed syndrome during the covid pandemic over  weeks   the study included  participants aged in their s and s who were randomly divided into two groups the experimental group received diaphragmatic breathing reeducation and shoulder stabilization exercises while the control group only underwent shoulder stabilization exercises both groups were trained three times a week for four weeks using telerehabilitation the comparison of withingroup prepost differences in the experimental and control groups was conducted using a paired ttest while the effects of treatment were assessed using repeatedmeasures analysis of var

### 특수문자, 숫자, '/n', '소제목' 제거

In [60]:
import re
def word_cleaning(raw):
    raw = raw.replace("\n"," ")
    raw = raw.replace("background:"," ")
    raw = raw.replace("purpose:"," ")
    raw = raw.replace("objectives:"," ")
    raw = raw.replace("objective:"," ")
    raw = raw.replace("design:"," ")
    raw = raw.replace("methods:"," ")
    raw = raw.replace("method:"," ")
    raw = raw.replace("results:"," ")
    raw = raw.replace("result:"," ")
    raw = raw.replace("discussion:"," ")
    raw = raw.replace("conclusions:"," ")
    raw = raw.replace("interpretation:"," ")
    raw = raw.replace("conclusion:"," ")
    raw = raw.replace("significance:"," ")
    raw = raw.replace("keywords:"," ")
    raw = raw.replace("fhp"," ")
    raw = raw.replace("forward head posture"," ")
    raw = raw.replace("head forward posture"," ")
    patterns = '([^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|a-z|A-Z|\'| ])'
    text_regex = re.sub(pattern = patterns, repl = "",string=raw)
    return text_regex

In [61]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_clean'].apply(word_cleaning)

In [62]:
FHP_article

,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join
0,Effects of scapular treatment on chronic neck ...,2024 Apr 1,Background: Chronic neck pain (CNP) is a commo...,2024,Apr,1,chronic neck pain cnp is a common public hea...,"[chronic, neck, pain, cnp, is, common, public,...",chronic neck pain cnp is common public health ...
1,Association between smartphone addiction and m...,2024 Apr 1,Background: The purpose of this study was to c...,2024,Apr,1,the purpose of this study was to clarify the...,"[purpose, study, was, clarify, relationship, s...",purpose study was clarify relationship smartph...
2,Effects of Telerehabilitation Combining Diaphr...,2024 Mar 11,Background: Forward head posture and rounded s...,2024,Mar,11,and rounded shoulder posture are common po...,"[head, posture, rounded, shoulder, posture, ar...",head posture rounded shoulder posture are comm...
3,Effects of Myofascial Release Technique along ...,2024 Mar 4,The purpose of this randomized controlled tria...,2024,Mar,4,the purpose of this randomized controlled tria...,"[purpose, randomized, controlled, trial, was, ...",purpose randomized controlled trial was evalua...
4,Sensory Organization and Postural Control Stra...,2024 Mar 26,Objective: This study aimed to compare sensory...,2024,Mar,26,this study aimed to compare sensory organiza...,"[study, aimed, compare, sensory, organization,...",study aimed compare sensory organization test ...
...,...,...,...,...,...,...,...,...,...
1246,Mechanism of production of gait unsteadiness b...,1975 Feb,The hypothesis is put forward that the disturb...,1975,Feb,None,the hypothesis is put forward that the disturb...,"[hypothesis, is, put, disturbance, gait, seen,...",hypothesis is put disturbance gait seen tumour...
1247,Effect of forward head inclination on visual o...,1972 Nov,Effect of forward head inclination on visual o...,1972,Nov,None,effect of forward head inclination on visual o...,"[effect, forward, head, inclination, visual, o...",effect forward head inclination visual orienta...
1248,[The mechanism of elongation of the eye-axis. ...,1972 Aug,[The mechanism of elongation of the eye-axis. ...,1972,Aug,None,the mechanism of elongation of the eyeaxis i t...,"[mechanism, elongation, eyeaxis, i, intraocula...",mechanism elongation eyeaxis i intraocular pre...
1249,Rheumatoid arthritis--atlanto-axial subluxatio...,1966 Mar 5,"The clinical picture, radiological findings an...",1966,Mar,5,the clinical picture radiological findings and...,"[clinical, picture, radiological, findings, tr...",clinical picture radiological findings treatme...


### 단어 토큰화

In [273]:
def word_tokenizing(raw):
    tokenizer = RegexpTokenizer("[\s]+", gaps=True) 
    tokens = tokenizer.tokenize(raw)
    tags = nltk.pos_tag(tokens)
    # nouns_list = [t for t in tags if t[1] in  ['NN','NNP','NNS']]
    nouns_list = [t for t in tags if t[1] in  ['JJ','JJR','NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    # nouns_list = [t for t in tags if t[1] in  ['NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    return nouns_list
# def word_tokenizing(raw):
#     tokenizer = RegexpTokenizer("[\s]+", gaps=True) 
#     text_list = []
#     for raw_senetence in raw:
#         tokens = tokenizer.tokenize(raw_senetence)
#         tags = nltk.pos_tag(tokens)
#         text_list.append(nouns_list)
#     return text_list

<>:2: DeprecationWarning: invalid escape sequence '\s'
<>:2: DeprecationWarning: invalid escape sequence '\s'
/tmp/ipykernel_9117/1482873834.py:2: DeprecationWarning: invalid escape sequence '\s'
  tokenizer = RegexpTokenizer("[\s]+", gaps=True)


In [274]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_clean'].apply(word_tokenizing)

In [275]:
FHP_article['abstract_token']

0                                                                                                              [(chronic, JJ), (neck, NN), (pain, NN), (cnp, NN), (is, VBZ), (common, JJ), (public, JJ), (health, NN), (problem, NN), (affects, VBZ), (daily, JJ), (living, NN), (activities, NNS), (quality, NN), (life, NN), (is, VBZ), (biomechanical, JJ), (interdependence, NN), (neck, NN), (scapula, NN), (studies, NNS), (have, VBP), (shown, VBN), (shoulder, NN), (blade, NN), (function, NN), (be, VB), (related, VBN), (chronic, JJ), (neck, NN), (pain, NN), (evaluated, VBD), (effects, NNS), (scapular, JJ), (targeted, VBN), (therapy, NN), (neck, NN), (pain, NN), (function, NN), (patients, NNS), (cnp, JJ), (databases, NNS), (including, VBG), (medline, NN), (pubmed, JJ), (embase, NN), (ovid, NN), (ovid, JJ), (web, NN), (science, NN), (scopus, NN), (were, VBD), (searched, VBN), (randomized, VBN), (controlled, JJ), (trials, NNS), (published, VBN), (english, JJ), (investigating, VBG), (treatment, NN),

### 표제어 추출

In [276]:
def Lemmatization(raw):
    lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    for word in raw:
        if word[1][0].lower() in ['v','n']:
            lemmatized_word = lemmatizer.lemmatize(word[0],word[1][0].lower())
            if lemmatized_word != 'be':
                lemmatized_list.append(lemmatized_word)
        else:
            lemmatized_list.append(word[0])
    return lemmatized_list
# def Lemmatization(raw):
#     lemmatizer = WordNetLemmatizer()
#     text_list = []
#     for raw_senetence in raw:
#         lemmatized_list = []
#         for word in raw_senetence:
#             if word[1][0].lower() in ['v','n']:
#                 lemmatized_word = lemmatizer.lemmatize(word[0],word[1][0].lower())
#                 lemmatized_list.append(lemmatized_word)
#         text_list.append(lemmatized_list)
#     return text_list

In [277]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_token'].apply(Lemmatization)

In [278]:
FHP_article.loc[:,'abstract_token']

0                                                                                                        [chronic, neck, pain, cnp, common, public, health, problem, affect, daily, living, activity, quality, life, biomechanical, interdependence, neck, scapula, study, have, show, shoulder, blade, function, relate, chronic, neck, pain, evaluate, effect, scapular, target, therapy, neck, pain, function, patient, cnp, database, include, medline, pubmed, embase, ovid, ovid, web, science, scopus, search, randomize, controlled, trial, publish, english, investigate, treatment, scapula, cnp, july, total, participant, include, rcts, compare, control, group, intervention, scapular, treatment, group, exhibit, greater, improvement, pain, intensity, standardize, mean, difference, smd, ci, p, moderate, evidence, subgroup, analysis, pain, intensity, reveal, significant, difference, sex, female, population, smd, ci, show, better, outcome, sex, smd, ...]
1                                                  

In [279]:
def join_words(raw):
    sentence = ' '.join(raw)
    return sentence
# def join_words(raw):
#     text_list = []
#     for raw_senetence in raw:
#         sentence = ' '.join(raw_senetence)
#         text_list.append(sentence)
#     return text_list

In [280]:
FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_token'].apply(join_words)

In [281]:
FHP_article

,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
0,Effects of scapular treatment on chronic neck pain: a systematic review and meta-analysis of randomized controlled trials,2024 Apr 1,"Background: Chronic neck pain (CNP) is a common public health problem that affects daily living activities and quality of life. There is biomechanical interdependence between the neck and scapula. Studies have shown that shoulder blade function might be related to chronic neck pain. We therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with CNP.\nMethods: Databases, including MEDLINE (via PubMed), EMBASE (via Ovid), Ovid, Web of Science, and Scopus, were systematically searched for randomized controlled trials published in English investigating treatment of the scapula for CNP before July 16, 2023.\nResults: A total of 313 participants were included from 8 RCTs. Compared with those in the control group, the intervention in the scapular treatment group exhibited greater improvement in pain intensity (standardized mean difference (SMD) = 2.55; 95% CI = 0.97 to 4.13; P = 0.002), with moderate evidence. Subgroup analysis for pain intensity revealed a significant difference between the sexes, with only the female population (SMD = 6.23, 95% CI = 4.80 to 7.65) showing better outcomes than those with both sexes (SMD = 1.07, 95% CI = 0.57 to 1.56) (p < 0.00001). However, moderate evidence demonstrated no improvement in neck disability after scapular treatment (SMD of 0.24[-0.14, 0.62] of Neck Disability Index or Northwick Park Neck Pain Questionnaire). No effect of scapular treatment was shown on the pressure pain threshold (PPT). The cervical range of motion (CROM) and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles, and further study was needed. However, the patient's head forward posture appeared to be corrected after scapular treatment.\nConclusion: Scapular therapy was beneficial for relieving pain intensity in patients with CNP, especially in women. Head forward posture might also be corrected with scapular therapy. However, scapular therapy may have no effect on the PPT or neck disability. However, whether scapular therapy could improve CROM and cervical muscle activation in patients with CNPs had not been determined and needed further study.",2024,Apr,1,chronic neck pain cnp is a common public health problem that affects daily living activities and quality of life there is biomechanical interdependence between the neck and scapula studies have shown that shoulder blade function might be related to chronic neck pain we therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with cnp databases including medline via pubmed embase via ovid ovid web of science and scopus were systematically searched for randomized controlled trials published in english investigating treatment of the scapula for cnp before july a total of participants were included from rcts compared with those in the control group the intervention in the scapular treatment group exhibited greater improvement in pain intensity standardized mean difference smd ci to p with moderate evidence subgroup analysis for pain intensity revealed a significant difference between the sexes with only the female population smd ci to showing better outcomes than those with both sexes smd ci to p however moderate evidence demonstrated no improvement in neck disability after scapular treatment smd of of neck disability index or northwick park neck pain questionnaire no effect of scapular treatment was shown on the pressure pain threshold ppt the cervical range of motion crom and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles and further study was needed however the patient's appeared to be corrected after scapular 

### 유사한 단어 묶기

In [282]:
def word_cleaning(raw):
    raw = raw.replace("chronic neck pain","cnp")
    return raw

## 학습데이터, 테스트 데이터 분류

In [283]:
from sklearn.model_selection import train_test_split
FHP_article_train,FHP_article_test = train_test_split(FHP_article,test_size=0.4,random_state=42)

## 키워드 분류하기

### TfidfVectorizer 최적화

In [306]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(max_df=0.60,min_df=0.2,ngram_range=(1,2), stop_words = ['use','have','show'])
features_train = tfidfVectorizer.fit_transform(FHP_article_train['abstract_join'])
dictionary_list = tfidfVectorizer.get_feature_names_out()
dictionary_list

array(['activity', 'age', 'aim', 'analysis', 'angle', 'assess', 'body',
       'cervical', 'change', 'compare', 'control', 'difference',
       'different', 'effect', 'evaluate', 'find', 'forward', 'group',
       'include', 'increase', 'investigate', 'measure', 'movement',
       'muscle', 'neck', 'pain', 'patient', 'perform', 'position',
       'postural', 'posture', 'result', 'significant', 'subject',
       'suggest', 'system', 'test', 'year'], dtype=object)

### gridsearch

In [307]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

# LDA 모델을 위한 파라미터 그리드 생성
param_grid = {
    'n_components': [3],
    'learning_decay': [0.7],
    'doc_topic_prior': [1.0],  # 문서-주제 사전확률 (알파)
    'topic_word_prior': [1.0]  # 단어-주제 사전확률 (베타)
}

# LatentDirichletAllocation 모델 인스턴스 생성
lda = LatentDirichletAllocation()

# GridSearchCV 인스턴스 생성
model = GridSearchCV(lda, param_grid, n_jobs=-1, cv=5)

# 데이터는 여기에 당신의 데이터셋을 사용하세요
# X는 문서-단어 행렬이어야 합니다. 예를 들어, CountVectorizer나 TfidfVectorizer를 사용하여 생성할 수 있습니다.
model.fit(features_train)

# 최적 모델
grid_best_model = model.best_estimator_

# 최적의 파라미터와 점수를 출력
print("Best Model's Params: ", model.best_params_)
print("Best Log Likelihood Score: ", model.best_score_)
print("perplexity",grid_best_model.perplexity(features_train) )


Best Model's Params:  {'doc_topic_prior': 1.0, 'learning_decay': 0.7, 'n_components': 3, 'topic_word_prior': 1.0}
Best Log Likelihood Score:  -1694.6914642653314
perplexity 49.37958635371941


In [308]:
grid_best_model.fit(features_train)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
df_topics_keywords_grid=df_topics_keywords_grid.T
df_topics_keywords_grid

,0,1
0,Topic1,group angle posture body pain significant difference measure age change
1,Topic2,position subject movement test system posture forward control body result
2,Topic3,muscle neck patient cervical postural activity posture pain change compare


In [309]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features_train,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [310]:
topics_output = grid_best_model.transform(features_train)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
1    259
2    249
0    242
Name: count, dtype: int64

### test set에 적용해보기

In [311]:
features_test = tfidfVectorizer.fit_transform(FHP_article_test['abstract_join'])
grid_best_model.fit(features_test)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
df_topics_keywords_grid=df_topics_keywords_grid.T
df_topics_keywords_grid

,0,1
0,Topic1,neck body muscle pain patient group analysis postural compare change
1,Topic2,postural group position assess compare patient different movement analysis significant
2,Topic3,perform subject suggest cervical postural result effect forward movement measure


In [312]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features_test,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [313]:
topics_output = grid_best_model.transform(features_test)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
2    181
0    178
1    142
Name: count, dtype: int64

### 전체 데이터에 적용

In [314]:
features = tfidfVectorizer.fit_transform(FHP_article['abstract_join'])
grid_best_model.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
df_topics_keywords_grid=df_topics_keywords_grid.T
df_topics_keywords_grid

,0,1
0,Topic1,group muscle pain compare difference result analysis body significant postural
1,Topic2,posture position neck body analysis movement cervical increase subject forward
2,Topic3,patient assess subject postural movement forward test posture cervical increase


In [342]:
df_datas_0 = [grid_best_model.components_[0], dictionary_list]
df_topics_0 = pd.DataFrame(data=df_datas_0).T
df_topics_0 = df_topics_0.sort_values(0, ascending=False).reset_index()
df_topics_0.rename(columns = {1 : 'first'}, inplace = True)
df_topics_0.rename(columns = {0 : 'count'}, inplace = True)
df_datas_1 = [grid_best_model.components_[1], dictionary_list]
df_topics_1 = pd.DataFrame(data=df_datas_1).T
df_topics_1 = df_topics_1.sort_values(0, ascending=False).reset_index()
df_topics_1.rename(columns = {1 : 'second'}, inplace = True)
df_topics_1.rename(columns = {0 : 'count'}, inplace = True)
df_datas_2 = [grid_best_model.components_[2], dictionary_list]
df_topics_2 = pd.DataFrame(data=df_datas_2).T
df_topics_2 = df_topics_2.sort_values(0, ascending=False).reset_index()
df_topics_2.rename(columns = {1 : 'third'}, inplace = True)
df_topics_2.rename(columns = {0 : 'count'}, inplace = True)
df_topic = pd.concat([df_topics_0.loc[:20,['count','first']],df_topics_1.loc[:20,['count','second']],df_topics_2.loc[:20,['count','third']]],axis=1)
df_topic

,count,first,count,second,count,third
0,113.300723,group,111.126228,posture,93.58349,patient
1,104.273278,muscle,108.487814,position,81.465812,assess
2,78.105097,pain,91.138433,neck,62.988489,subject
3,56.405029,compare,57.533385,body,61.310274,postural
4,45.106438,difference,53.082659,analysis,47.34251,movement
5,43.967451,result,44.08325,movement,42.584857,forward
6,42.00627,analysis,39.901196,cervical,40.627076,test
7,38.512706,body,39.888425,increase,40.018472,posture
8,37.173591,significant,33.003525,subject,39.185409,cervical
9,36.708832,postural,32.623565,forward,39.073513,increase


In [343]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features,tfidfVectorizer)
# pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [344]:
topics_output = grid_best_model.transform(features)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
0    441
2    408
1    402
Name: count, dtype: int64

## 토픽 분석하기

### topic별로 분류하기

In [321]:
# FHP_article = FHP_article_train.reset_index()
FHP_article['article_topic'] = df_topics_score['dominant_topic_number']

In [322]:
FHP_article

,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
0,Effects of scapular treatment on chronic neck pain: a systematic review and meta-analysis of randomized controlled trials,2024 Apr 1,"Background: Chronic neck pain (CNP) is a common public health problem that affects daily living activities and quality of life. There is biomechanical interdependence between the neck and scapula. Studies have shown that shoulder blade function might be related to chronic neck pain. We therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with CNP.\nMethods: Databases, including MEDLINE (via PubMed), EMBASE (via Ovid), Ovid, Web of Science, and Scopus, were systematically searched for randomized controlled trials published in English investigating treatment of the scapula for CNP before July 16, 2023.\nResults: A total of 313 participants were included from 8 RCTs. Compared with those in the control group, the intervention in the scapular treatment group exhibited greater improvement in pain intensity (standardized mean difference (SMD) = 2.55; 95% CI = 0.97 to 4.13; P = 0.002), with moderate evidence. Subgroup analysis for pain intensity revealed a significant difference between the sexes, with only the female population (SMD = 6.23, 95% CI = 4.80 to 7.65) showing better outcomes than those with both sexes (SMD = 1.07, 95% CI = 0.57 to 1.56) (p < 0.00001). However, moderate evidence demonstrated no improvement in neck disability after scapular treatment (SMD of 0.24[-0.14, 0.62] of Neck Disability Index or Northwick Park Neck Pain Questionnaire). No effect of scapular treatment was shown on the pressure pain threshold (PPT). The cervical range of motion (CROM) and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles, and further study was needed. However, the patient's head forward posture appeared to be corrected after scapular treatment.\nConclusion: Scapular therapy was beneficial for relieving pain intensity in patients with CNP, especially in women. Head forward posture might also be corrected with scapular therapy. However, scapular therapy may have no effect on the PPT or neck disability. However, whether scapular therapy could improve CROM and cervical muscle activation in patients with CNPs had not been determined and needed further study.",2024,Apr,1,chronic neck pain cnp is a common public health problem that affects daily living activities and quality of life there is biomechanical interdependence between the neck and scapula studies have shown that shoulder blade function might be related to chronic neck pain we therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with cnp databases including medline via pubmed embase via ovid ovid web of science and scopus were systematically searched for randomized controlled trials published in english investigating treatment of the scapula for cnp before july a total of participants were included from rcts compared with those in the control group the intervention in the scapular treatment group exhibited greater improvement in pain intensity standardized mean difference smd ci to p with moderate evidence subgroup analysis for pain intensity revealed a significant difference between the sexes with only the female population smd ci to showing better outcomes than those with both sexes smd ci to p however moderate evidence demonstrated no improvement in neck disability after scapular treatment smd of of neck disability index or northwick park neck pain questionnaire no effect of scapular treatment was shown on the pressure pain threshold ppt the cervical range of motion crom and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles and further study was needed however the patient's appeared to be corrected after scapular 

### topic 별 많이 등장하는 단어 추출

In [336]:
FHP_article.query("article_topic == 0")

,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
0,Effects of scapular treatment on chronic neck pain: a systematic review and meta-analysis of randomized controlled trials,2024 Apr 1,"Background: Chronic neck pain (CNP) is a common public health problem that affects daily living activities and quality of life. There is biomechanical interdependence between the neck and scapula. Studies have shown that shoulder blade function might be related to chronic neck pain. We therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with CNP.\nMethods: Databases, including MEDLINE (via PubMed), EMBASE (via Ovid), Ovid, Web of Science, and Scopus, were systematically searched for randomized controlled trials published in English investigating treatment of the scapula for CNP before July 16, 2023.\nResults: A total of 313 participants were included from 8 RCTs. Compared with those in the control group, the intervention in the scapular treatment group exhibited greater improvement in pain intensity (standardized mean difference (SMD) = 2.55; 95% CI = 0.97 to 4.13; P = 0.002), with moderate evidence. Subgroup analysis for pain intensity revealed a significant difference between the sexes, with only the female population (SMD = 6.23, 95% CI = 4.80 to 7.65) showing better outcomes than those with both sexes (SMD = 1.07, 95% CI = 0.57 to 1.56) (p < 0.00001). However, moderate evidence demonstrated no improvement in neck disability after scapular treatment (SMD of 0.24[-0.14, 0.62] of Neck Disability Index or Northwick Park Neck Pain Questionnaire). No effect of scapular treatment was shown on the pressure pain threshold (PPT). The cervical range of motion (CROM) and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles, and further study was needed. However, the patient's head forward posture appeared to be corrected after scapular treatment.\nConclusion: Scapular therapy was beneficial for relieving pain intensity in patients with CNP, especially in women. Head forward posture might also be corrected with scapular therapy. However, scapular therapy may have no effect on the PPT or neck disability. However, whether scapular therapy could improve CROM and cervical muscle activation in patients with CNPs had not been determined and needed further study.",2024,Apr,1,chronic neck pain cnp is a common public health problem that affects daily living activities and quality of life there is biomechanical interdependence between the neck and scapula studies have shown that shoulder blade function might be related to chronic neck pain we therefore evaluated the effects of scapular targeted therapy on neck pain and function in patients with cnp databases including medline via pubmed embase via ovid ovid web of science and scopus were systematically searched for randomized controlled trials published in english investigating treatment of the scapula for cnp before july a total of participants were included from rcts compared with those in the control group the intervention in the scapular treatment group exhibited greater improvement in pain intensity standardized mean difference smd ci to p with moderate evidence subgroup analysis for pain intensity revealed a significant difference between the sexes with only the female population smd ci to showing better outcomes than those with both sexes smd ci to p however moderate evidence demonstrated no improvement in neck disability after scapular treatment smd of of neck disability index or northwick park neck pain questionnaire no effect of scapular treatment was shown on the pressure pain threshold ppt the cervical range of motion crom and electromyographic activity of neck muscles could not be conclusively evaluated due to limited support in the articles and further study was needed however the patient's appeared to be corrected after scapular 

In [337]:
FHP_article.query("article_topic == 1")

,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
1,Association between smartphone addiction and myofascial trigger points,2024 Apr 1,"Background: The purpose of this study was to clarify the relationship between smartphone addiction and miyafascial trigger points in university students.\nMethods: A cross-sectional study of university students was conducted for the purpose of this study. The participants were assessed based on age, gender, dominant side, the amount of time they spent on their smartphones, the purpose of their use, and their posture. The Smartphone Addiction Scale Short Form (SAS-SF) was used to determine addictes and non-addicts. The cut-off value of SAS-SF is 31 and above for male and 33 and above for female.\nResults: There were 136 participants in the study. The posture score for addicts and non-addicts ones was not significantly different (p > 0,05), but the number of trigger points, maximal bending posture and trigger points in the right levator scapula and right cervical erector muscles were significantly higher in the smartphone addict participants (p < 0,05).\nConclusions: Smartphone addiction in university students is associated with postural changes and trigger points in the bilateral levator scapula and right cervical erector muscles. Public health programs should be developed to raise awareness about smartphone addiction, encourage screen breaks, and emphasize physical activity and exercise regularly.",2024,Apr,1,the purpose of this study was to clarify the relationship between smartphone addiction and miyafascial trigger points in university students a crosssectional study of university students was conducted for the purpose of this study the participants were assessed based on age gender dominant side the amount of time they spent on their smartphones the purpose of their use and their posture the smartphone addiction scale short form sassf was used to determine addictes and nonaddicts the cutoff value of sassf is and above for male and and above for female there were participants in the study the posture score for addicts and nonaddicts ones was not significantly different p but the number of trigger points maximal bending posture and trigger points in the right levator scapula and right cervical erector muscles were significantly higher in the smartphone addict participants p smartphone addiction in university students is associated with postural changes and trigger points in the bilateral levator scapula and right cervical erector muscles public health programs should be developed to raise awareness about smartphone addiction encourage screen breaks and emphasize physical activity and exercise regularly,"[purpose, study, clarify, relationship, smartphone, addiction, miyafascial, trigger, point, university, student, crosssectional, study, university, student, conduct, purpose, study, participant, assess, base, age, gender, dominant, side, amount, time, spend, smartphones, purpose, use, posture, smartphone, addiction, scale, short, form, sassf, use, determine, addictes, nonaddicts, cutoff, value, sassf, male, above, female, participant, study, posture, score, addict, nonaddicts, one, different, p, number, trigger, point, maximal, bending, posture, trigger, point, right, levator, scapula, right, cervical, erector, muscle, higher, smartphone, addict, participant, p, smartphone, addiction, university, student, associate, postural, change, trigger, point, bilateral, levator, scapula, right, cervical, erector, muscle, public, health, program, develop, raise, awareness, smartphone, ...]",purpose study clarify relationship smartphone addiction miyafascial trigger point university student crosssectional study university student conduct purpose study participant assess base age gender dominant side amount time spend smartphones purpose use posture smartphone addiction scale short form sassf use determine addictes nonaddicts cutoff value sassf ma

In [338]:
FHP_article.query("article_topic == 2")

,title,date,abstract,date_year,date_month,date_date,abstract_clean,abstract_token,abstract_join,article_topic
14,[Effect of the forward head syndrome in the development of temporomandibular disorders],2023 Dec 26,"Bad habits, such as poor posture during the use of technological devices, poor abdominal control and oral breathing pattern can lead to a forward head position, which has important implications for the entire future of the individual, especially at the stomatognathic system. There is a strong association between temporomandibular disorders and forward head syndrome. Where the muscular component is the main affected, with the appearance of more trigger points at the level of the sternocleidomastoid muscles, upper trapezius, rectus capitis posterior and upper oblique capitis. Degenerative changes in the temporomandibular joint have been observed, but evidence is still lacking to ensure that Forward Head Syndrome is the etiological factor.",2023,Dec,26,bad habits such as poor posture during the use of technological devices poor abdominal control and oral breathing pattern can lead to a forward head position which has important implications for the entire future of the individual especially at the stomatognathic system there is a strong association between temporomandibular disorders and forward head syndrome where the muscular component is the main affected with the appearance of more trigger points at the level of the sternocleidomastoid muscles upper trapezius rectus capitis posterior and upper oblique capitis degenerative changes in the temporomandibular joint have been observed but evidence is still lacking to ensure that forward head syndrome is the etiological factor,"[bad, habit, such, poor, posture, use, technological, device, poor, abdominal, control, oral, breathing, pattern, lead, head, position, have, important, implication, entire, future, individual, stomatognathic, system, strong, association, temporomandibular, disorder, head, syndrome, muscular, component, main, affected, appearance, more, trigger, point, level, sternocleidomastoid, muscle, upper, trapezius, rectus, capitis, posterior, upper, oblique, capitis, degenerative, change, temporomandibular, joint, have, observe, evidence, lack, ensure, forward, head, syndrome, etiological, factor]",bad habit such poor posture use technological device poor abdominal control oral breathing pattern lead head position have important implication entire future individual stomatognathic system strong association temporomandibular disorder head syndrome muscular component main affected appearance more trigger point level sternocleidomastoid muscle upper trapezius rectus capitis posterior upper oblique capitis degenerative change temporomandibular joint have observe evidence lack ensure forward head syndrome etiological factor,2
16,The influence of jump-landing direction on dynamic postural stability following anterior cruciate ligament reconstruction,2024 Feb,"Background: Traditional testing prior to return to sport following anterior cruciate ligament reconstruction typically involves jump-landing tasks in the forward direction. As injury is most likely the result of multiplanar neuromuscular control deficits, assessment of dynamic postural stability using landing tasks that require multiplanar stabilization may be more appropriate. The purpose of this study was to examine how dynamic postural stability is affected when performing jump-landing tasks in three different directions.\nMethods: Fifteen athletes [11 females (18.0 ± 3.0 years) and 4 males (18.5 ± 3.1 years)] following anterior cruciate ligament reconstruction performed a series of single-limb jump-landing tasks in 3 directions. Individual directional stability indices and a composite dynamic postural stability index were calculated using ground reaction force data and were compared using separate one-way repeated measures ANOVAs.\nFindings: All directional stability indices demonstrated a significant main e

In [339]:
FHP_article.to_csv('FHP_article.csv',index=False,encoding='utf-8')

## 결론
- 토픽 별로 논문 초록을 확인해 본 결과, 거북목과 관련되지 않은 논문도 많이 존재함
- 논문을 검색할 때 거북목이 무조건 들어가 있는 논문만 결과로 나올 수 있게 하여 다시 시도 필요
  

### Count vectorizer

In [323]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words=[], ngram_range=(1, 2))

In [324]:
FHP_article.query("article_topic == 2")['abstract_join']

14                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [325]:
count_vect.fit(FHP_article['abstract_join'])

CountVectorizer(ngram_range=(1, 2), stop_words=[])

In [326]:
count_vect.vocabulary_

{'chronic': 15038,
 'neck': 64735,
 'pain': 69809,
 'cnp': 15984,
 'common': 16636,
 'public': 81016,
 'health': 43689,
 'problem': 79340,
 'affect': 2783,
 'daily': 22363,
 'living': 55912,
 'activity': 1387,
 'quality': 81368,
 'life': 55166,
 'biomechanical': 10592,
 'interdependence': 50804,
 'scapula': 89561,
 'study': 99048,
 'have': 42239,
 'show': 92670,
 'shoulder': 92477,
 'blade': 10776,
 'function': 39301,
 'relate': 84435,
 'evaluate': 32343,
 'effect': 29714,
 'scapular': 89608,
 'target': 102894,
 'therapy': 104320,
 'patient': 71386,
 'database': 22666,
 'include': 47787,
 'medline': 59902,
 'pubmed': 81046,
 'embase': 30862,
 'ovid': 69693,
 'web': 113633,
 'science': 89793,
 'scopus': 89910,
 'search': 90218,
 'randomize': 81950,
 'controlled': 20101,
 'trial': 107488,
 'publish': 81026,
 'english': 31407,
 'investigate': 51470,
 'treatment': 107229,
 'july': 52431,
 'total': 106245,
 'participant': 70817,
 'rcts': 82384,
 'compare': 16793,
 'control': 19771,
 'group'

In [327]:
dtm_topic_0 = count_vect.transform(FHP_article.query("article_topic == 0")['abstract_join'])
dtm_array_0 = dtm_topic_0.toarray()
df_dtm_0 = pd.DataFrame(dtm_array_0, columns = vocabs)
df_first_topic = df_dtm_0.sum().sort_values(ascending=False)[:20]
df_first_topic = df_first_topic.reset_index()
df_first_topic.rename(columns = {'index' : 'first'}, inplace = True)
df_first_topic.rename(columns = {0 : 'count'}, inplace = True)

(441, 115337)

In [328]:
dtm_topic_1 = count_vect.transform(FHP_article.query("article_topic == 1")['abstract_join'])
dtm_array_1 = dtm_topic_1.toarray()
df_dtm_1 = pd.DataFrame(dtm_array_1, columns = vocabs)
df_second_topic = df_dtm_1.sum().sort_values(ascending=False)[:20]
df_second_topic = df_second_topic.reset_index()
df_second_topic.rename(columns = {'index' : 'second'}, inplace = True)
df_second_topic.rename(columns = {0 : 'count'}, inplace = True)

(402, 115337)

In [329]:
dtm_topic_2 = count_vect.transform(FHP_article.query("article_topic == 2")['abstract_join'])
dtm_array_2 = dtm_topic_2.toarray()
df_dtm_2 = pd.DataFrame(dtm_array_2, columns = vocabs)
df_third_topic = df_dtm_2.sum().sort_values(ascending=False)[:20]
df_third_topic = df_third_topic.reset_index()
df_third_topic.rename(columns = {'index' : 'third'}, inplace = True)
df_third_topic.rename(columns = {0 : 'count'}, inplace = True)

(408, 115337)

In [330]:
vocabs = count_vect.get_feature_names_out()
vocabs[:2]
vocabs.shape

(115337,)

In [334]:
df_topic = pd.concat([df_first_topic,df_second_topic,df_third_topic],axis=1)

In [335]:
df_topic

,first,count,second,count,third,count
0,group,1253,head,1058,head,899
1,muscle,876,posture,992,patient,583
2,pain,824,position,920,body,555
3,study,746,neck,636,subject,505
4,neck,708,study,527,postural,456
5,head,586,use,523,movement,389
6,posture,559,cervical,451,study,388
7,use,539,angle,418,response,369
8,cervical,538,have,386,use,360
9,control,503,increase,297,have,357
